In [1]:
pip install langchain
pip install --upgrade langchain_core
pip install --upgrade sqlalchemy
# pip install prompts_chat_pdf Not USING THIS
pip install -q pypdf
pip install -q sentence-transformers
pip install -q faiss-cpu
pip install ctransformers

  Obtaining dependency information for langchain_core from https://files.pythonhosted.org/packages/25/d9/c22eb2606e9c77eeb99099dfc9a114e848386227e123ac4fb14a0ed954c6/langchain_core-0.1.18-py3-none-any.whl.metadata
  Using cached langchain_core-0.1.18-py3-none-any.whl.metadata (6.0 kB)
  Obtaining dependency information for jsonpatch<2.0,>=1.33 from https://files.pythonhosted.org/packages/73/07/02e16ed01e04a374e644b575638ec7987ae846d25ad97bcc9945a3ee4b0e/jsonpatch-1.33-py2.py3-none-any.whl.metadata
  Using cached jsonpatch-1.33-py2.py3-none-any.whl.metadata (3.0 kB)
  Obtaining dependency information for langsmith<0.1,>=0.0.83 from https://files.pythonhosted.org/packages/67/a0/a43239e2a0fa12867c66ef3f40537afb1d23c2830fef8bb624f578bf35e2/langsmith-0.0.86-py3-none-any.whl.metadata
  Using cached langsmith-0.0.86-py3-none-any.whl.metadata (10 kB)
  Obtaining dependency information for packaging<24.0,>=23.2 from https://files.pythonhosted.org/packages/ec/1a/610693ac4ee14fcdf2d9bf3c493370e4f

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tables 3.8.0 requires blosc2~=2.0.0, which is not installed.
tables 3.8.0 requires cython>=0.29.21, which is not installed.
python-lsp-black 1.2.1 requires black>=22.3.0, but you have black 0.0 which is incompatible.


In [ ]:
import os
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import ConversationalRetrievalChain
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.llms import CTransformers
# import chainlit as cl
from langchain.memory import ConversationBufferMemory


# from prompts_chat_pdf import chat_prompt, CONDENSE_QUESTION_PROMPT

In [2]:

class PDFChatBot:

    def __init__(self):
        self.data_path = os.path.join('data')
        self.db_faiss_path = os.path.join('vectordb', 'db_faiss')
        #self.chat_prompt = PromptTemplate(template=chat_prompt, input_variables=['context', 'question'])
        #self.CONDENSE_QUESTION_PROMPT=CONDENSE_QUESTION_PROMPT

    def create_vector_db(self):

        '''function to create vector db provided the pdf files'''

        loader = DirectoryLoader(self.data_path,
                             glob='*.pdf',
                             loader_cls=PyPDFLoader)

        documents = loader.load()
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=400,
                                                   chunk_overlap=50)
        texts = text_splitter.split_documents(documents)

        embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2',
                                       model_kwargs={'device': 'cpu'})

        db = FAISS.from_documents(texts, embeddings)
        db.save_local(self.db_faiss_path)

    def load_llm(self):
        # Load the locally downloaded model here
        llm = CTransformers(
            model="llama-2-7b.ggmlv3.q4_1.bin",
            model_type="llama",
            max_new_tokens=2000,
            temperature=0.5
        )
        return llm

    def conversational_chain(self):

        embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2",
                                           model_kwargs={'device': 'cpu'})
        db = FAISS.load_local(self.db_faiss_path, embeddings)
        # initializing the conversational chain
        memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
        conversational_chain = ConversationalRetrievalChain.from_llm( llm=self.load_llm(),
                                                                      retriever=db.as_retriever(search_kwargs={"k": 3}),
                                                                      verbose=True,
                                                                      memory=memory
                                                                      )

        return conversational_chain



In [3]:
def intialize_chain():
    bot = PDFChatBot()
    bot.create_vector_db()
    conversational_chain = bot.conversational_chain()
    return conversational_chain

chat_history = []

chain = intialize_chain()

await chain.arun("what is over fitting?")
# @cl.on_chat_start
# async def start():
#     msg = cl.Message(content="Starting the bot...")
#     await msg.send()
#     msg.content = "Welcome to Data Science Interview Prep Bot. Ask me your question?"
#     await msg.update()

#     cl.user_session.set("chain", chain)


# @cl.on_message
# async def main(message):
#     chain = cl.user_session.get("chain")
#     cb = cl.AsyncLangchainCallbackHandler(
#         stream_final_answer=False, answer_prefix_tokens=["FINAL", "ANSWER"]
#     )
#     cb.answer_reached = True
#     res = await chain.acall({"question": message, "chat_history": chat_history}, callbacks=[cb])
#     answer = res["answer"]
#     chat_history.append(answer)
#     await cl.Message(content=answer).send()

#     # while(True):
#     #     query = input('User: ')
#     #     response = conversational_chain({"question": query, "chat_history": chat_history})
#     #     chat_history.append(response["answer"])  # Append the answer to chat history
#     #     print(response["answer"])

C:\Users\nirma\anaconda3\Lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
C:\Users\nirma\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `arun` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use ainvoke instead.
  warn_deprecated(




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

fit than complex ones.
 A simple model  in this context is a model where the distribution of parameter values
has less entropy (or a model with fewer para meters, as you saw in the previous sec-
tion). Thus a common way to mitigate overfi tting is to put constraints on the complex-
ity of a network by forcing its weights to take only small values, which makes the

the model on the validation data. In essence, this tuning is a form of learning : a search
for a good configuration in some parameter space. As a result, tuning the configura-
tion of the model based on its performance on the validation set can quickly result in
overfitting to the validation set , even though your model is never directly trained on it.

began t

' An ML algorithm gets "tuned" in such a way as to make it fit better than necessary, i.e., not generalize as well on out of sample data'